In [2]:
import os
import pandas as pd
import gff3
import glob

In [3]:
### This script convert a simple list of gene names to BSU numbers

In [20]:
file = '/Users/andreas/Bacillus/Bioinformatics/Scripts_RNAseq/subtiwiki.gene.export.2024-05-25.tsv'
subti = pd.read_csv(file, sep = '\t', header = 0)


file = '/Users/andreas/Bacillus/Bioinformatics/RNAseq_jag_khpa_kre/X204SC22071444-Z01-F001/Analysis/Annotation_files/B_subtilis_GenBank_w_name_and_locus.gff3'
genbank = pd.read_csv(file, sep = '\t', header = 0, comment = '#')

### some genes are on the form geneX_geneY
### split these genes into two rows; geneX and geneY

input_folder = '/Users/andreas/Bacillus/Bioinformatics/CRAC_jag_khpA_kre_30thMay2024/DPW/z_scores/Top_CRAC_hits'
output_folder = '/Users/andreas/Bacillus/Bioinformatics/CRAC_jag_khpA_kre_30thMay2024/DPW/z_scores/Top_CRAC_hits/BSU'


for file in glob.glob(input_folder + '/*.txt'):
    df = pd.read_csv(file, sep = '\t', header = 0)
    gene_base = []
    for gene in df['Gene'].tolist():
        gene_base.append(gene.split('.')[0])
    df['Gene_base'] = gene_base
    df = df.drop_duplicates(['Gene_base'])
    df.to_csv(file, sep = '\t', header = True, index = False)

for file in glob.glob(input_folder + '/*.txt'):
    all = pd.read_csv(file, sep = '\t', header = 0)

    ### replace 'BSU_' with 'BSU^' 
    ### This way we can seperate transcripts by _ followingly
    ### replace 'BSU_' with 'BSU^' 
    ### This way we can seperate transcripts by _ followingly
    temp_gene_names = []
    for name in all['Gene_base'].tolist():
        temp_gene_names.append(name.replace('BSU_', 'BSU^').replace('NP_', 'NP^').replace('YP_', 'YP^').replace('Novel_transcript', 'Novel^transcript'))
    all['Gene_base'] = temp_gene_names

    _df = pd.DataFrame()

    # Loop through each row in the original DataFrame
    for index, row in all.iterrows():
        # Check if 'Gene_base' contains an underscore
        if '_' in row['Gene_base']:
            # Split the 'Gene_base' value into a list of genes
            gene_list = row['Gene_base'].split('_')
            # Create new rows for each gene in gene_list
            for gene in gene_list:
                new_row = pd.DataFrame({
                    'Gene': [gene],
                    'Gene_base': [gene]
                })
                _df = pd.concat([_df, new_row], ignore_index=True)
        ### replace '@' with ' ' in gene names
        elif '@' in row['Gene_base']:
            new_row = pd.DataFrame({
                    'Gene_base': [row['Gene_base'].replace('@', ' ')]
                })
            _df = pd.concat([_df, new_row], ignore_index=True)

        else:
            # If nothing else, keep the row as is
            _df = pd.concat([_df, all[all.index == index]], ignore_index=True)
    
    ### replace 'BSU_' with 'BSU^' 
    ### This way we can seperate transcripts by _ followingly
    revert_gene_names = []
    for name in _df['Gene_base'].tolist():
        revert_gene_names.append(name.replace('BSU^', 'BSU_').replace('NP^', 'NP_').replace('YP^', 'YP_').replace('Novel^transcript', 'Novel_transcript'))
    _df['Gene_base'] = revert_gene_names


    ### convert gene names to BSU_numbers using SubtiWiki and Genbank
    bsu_numbers = []
    for gene in _df['Gene_base'].tolist():
        try:
            bsu_numbers.append(subti['locus'][subti['title'] == gene].iloc[0])
            continue
        except IndexError:
            x = 0
            for index, row in subti.iterrows():
                try:
                    if gene in row['synonyms']:
                        bsu_numbers.append(subti['locus'][subti.index == index].iloc[0])
                        x += 1
                        break
                ## for na values
                except TypeError:
                    pass
            if x == 0:
                try:
                    bsu_numbers.append(genbank['locusTag'][genbank['name'] == gene].iloc[0])
                    continue
                except IndexError:
                    print(f'Error: Couldnt convert {gene}')
                    bsu_numbers.append('no_locusTag')
    
    _df['locusTag'] = bsu_numbers
    _df.drop_duplicates(['locusTag'])
    _df.to_csv(os.path.join(output_folder, os.path.basename(file)), sep = '\t', header = True, index = False)

Error: Couldnt convert YP_009514011
Error: Couldnt convert Novel_transcript


In [13]:
_df

,Gene,Gene_base
0,ypoP,ypoP
1,rpoC,rpoC
2,resA,resA
3,ybgB,ybgB
4,gltT,gltT
5,folEB,folEB
6,ypuD,ypuD
7,ftsE,ftsE
8,rpsR,rpsR
9,nfeDB,nfeDB


In [31]:
genbank

,seqID,source,feature,start,end,score,strand,phase,attributes,name,locusTag
0,NC_000964.3,RefSeq,region,1,4215606,.,+,.,ID=NC_000964.3:1..4215606;Dbxref=taxon:224308;...,no_name,no_locusTag
1,NC_000964.3,RefSeq,gene,410,1750,.,+,.,ID=gene-BSU_00010;Dbxref=GeneID:939978;Name=dn...,dnaA,BSU_00010
2,NC_000964.3,RefSeq,CDS,410,1750,.,+,0,ID=cds-NP_387882.1;Parent=gene-BSU_00010;Dbxre...,NP_387882.1,BSU_00010
3,NC_000964.3,RefSeq,gene,1939,3075,.,+,.,ID=gene-BSU_00020;Dbxref=GeneID:939970;Name=dn...,dnaN,BSU_00020
4,NC_000964.3,RefSeq,CDS,1939,3075,.,+,0,ID=cds-NP_387883.1;Parent=gene-BSU_00020;Dbxre...,NP_387883.1,BSU_00020
...,...,...,...,...,...,...,...,...,...,...,...
9283,NC_000964.3,RefSeq,CDS,4213823,4214608,.,-,0,ID=cds-NP_391984.1;Parent=gene-BSU_41040;Dbxre...,NP_391984.1,BSU_41040
9284,NC_000964.3,RefSeq,gene,4214753,4215103,.,-,.,ID=gene-BSU_41050;Dbxref=GeneID:937930;Name=rn...,rnpA,BSU_41050
9285,NC_000964.3,RefSeq,CDS,4214753,4215103,.,-,0,ID=cds-NP_391985.1;Parent=gene-BSU_41050;Dbxre...,NP_391985.1,BSU_41050
9286,NC_000964.3,RefSeq,gene,4215255,4215389,.,-,.,ID=gene-BSU_41060;Dbxref=GeneID:937942;Name=rp...,rpmH,BSU_41060
